In [1]:
import os

import numpy as np
import tensorflow as tf
tf.enable_eager_execution()


os.environ["TF_CPP_MIN_LOG_LEVEL"] = '2'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = ""

tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True
tf_config.gpu_options.per_process_gpu_memory_fraction = 0.3

In [2]:
%load_ext autoreload
%autoreload 2


import margin_loss
from mnist import data_provider as mnist
from mnist import mnist_config
from mnist import mnist_model

In [3]:
config = mnist_config.ConfigDict()
dataset = mnist.MNIST(
    data_dir="/home/xyang2/project/data/dataset/mnist",
    subset="train",
    batch_size=256,
    is_training=False)
model = mnist_model.MNISTNetwork(config)

(32, 1, 5, 5) 12.210585
(64, 32, 5, 5) -0.29181427
(512, 3136) 9.986719
(10, 512) -1.8749985


In [4]:
images, labels, num_examples, num_classes = (dataset.images, dataset.labels, dataset.num_examples, dataset.num_classes)
images, labels = dataset.get_next()
images.numpy().sum()

25744.127

# tensorflow (eager)

In [5]:
def weight_init_tf(shape):
    fan_in = 0
    if len(shape) == 4:
        fan_in = shape[1] * shape[2] * shape[3]
    if len(shape) == 2:
        fan_in = shape[1]
        print(shape)
    if fan_in:
        s = 1.0 * np.sqrt(6.0 / fan_in)
        transpose = np.random.uniform(-s, s, shape).astype("float32")
    if len(shape) == 2:
        transpose = transpose.T
    if len(shape) == 4:
        transpose = np.transpose(transpose, axes=(2, 3, 1, 0))
    return transpose

In [6]:
tf.set_random_seed(1)
np.random.seed(2)
conv_layer = tf.layers.Conv2D(kernel_size=5, filters=2, strides=(1, 1), padding="same",
                                          activation=None, use_bias=False, 
                              kernel_initializer=tf.constant_initializer((weight_init_tf((2, 1, 5, 5)))))
tf_h = conv_layer(images)
print(tf_h.shape)
tf_h.numpy().sum()

(256, 28, 28, 2)


-58702.617

In [7]:
conv_layer.weights[0].numpy().sum()

-2.2851927

In [8]:
np.random.seed(1)
tf.set_random_seed(1)
fc_layer = tf.layers.Dense(10, activation=None, kernel_initializer=tf.constant_initializer(weight_init_tf((10, 784))))
tf_h = fc_layer(tf.reshape(images, [256, 784]))
print(tf_h.numpy().sum())
fc_layer.weights

(10, 784)
-372.48087


[<tf.Variable 'dense/kernel:0' shape=(784, 10) dtype=float32, numpy=
 array([[-0.01451812,  0.0238486 , -0.0420277 , ..., -0.01036794,
         -0.03331389,  0.01813593],
        [ 0.03854876, -0.02185452,  0.01421322, ..., -0.00674237,
         -0.02058679,  0.02875962],
        [-0.08746176,  0.06254676,  0.0796543 , ...,  0.01677754,
          0.04565297, -0.03981096],
        ...,
        [ 0.08314659, -0.05625169,  0.03469209, ..., -0.02271525,
          0.08669829, -0.05285747],
        [-0.03143757,  0.08221023, -0.0618306 , ...,  0.0015496 ,
         -0.00324916, -0.00451572],
        [ 0.08441833,  0.0423306 ,  0.0196766 , ...,  0.03165539,
          0.05910492,  0.07916736]], dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

In [9]:
np.random.seed(1)
tf.set_random_seed(1)
fc_layer = tf.layers.Dense(10, activation=None, kernel_initializer=tf.constant_initializer(weight_init_tf((10, 784))))
tf_h = fc_layer(tf.reshape(images, [256, 784]))
print(tf_h.numpy().sum())
fc_layer.weights[0].numpy().shape

(10, 784)
-372.48087


(784, 10)

In [10]:
tf_prob_vector = tf.nn.softmax(tf_h)

# pytorch


In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as nfunc
from torch.nn.parameter import Parameter

In [12]:
# convert tensor from tensorflow to tensor of pytorch
torch_x = images.numpy()
print("numpy value", torch_x.sum())
torch_x = torch.from_numpy(torch_x).permute(0, 3, 1, 2).contiguous()
print(torch_x.sum().item(), " compare to tensorflow's value", 25744.127)
print(torch_x.shape)
print("convert back to numpy", x.numpy().sum())

numpy value 25744.127
25744.130859375  compare to tensorflow's value 25744.127
torch.Size([256, 1, 28, 28])


NameError: name 'x' is not defined

In [13]:
def weights_init_kaiming(m):
    """
    initialize normal distribution weight matrix
    and set bias to 0
    :param m:
    :return:
    """
    class_name = m.__class__.__name__
    fan_in = 0
    if class_name.find('Conv') != -1:
        shape = m.weight.data.shape
        print(shape)
        fan_in = shape[1] * shape[2] * shape[3]
    if class_name.find('Linear') != -1:
        shape = m.weight.data.shape
        fan_in = shape[1]
        print(shape)
    if fan_in:
        s = 1.0 * np.sqrt(6.0 / fan_in)
        transpose = np.random.uniform(-s, s, m.weight.data.shape).astype("float32")
        tensor = torch.from_numpy(transpose)
        m.weight = Parameter(tensor, requires_grad=True)
        if m.bias is not None:
            m.bias.data.zero_()
    if class_name.find('BatchNorm') != -1:
        transpose = np.ones(m.weight.data.shape).astype("float32")
        tensor = torch.from_numpy(transpose)
        m.weight = Parameter(tensor, requires_grad=True)
        if m.bias is not None:
            m.bias.data.zero_()

In [14]:
np.random.seed(1)
layer = nn.Conv2d(1, 2, kernel_size=5, stride=(1, 1), padding=2, bias=False)
layer.apply(weights_init_kaiming)
torch_h = layer(torch_x)
print(torch_h.shape)
torch_h.sum()

torch.Size([2, 1, 5, 5])
torch.Size([256, 2, 28, 28])


tensor(-41546.7070, grad_fn=<SumBackward0>)

In [15]:
layer.weight

Parameter containing:
tensor([[[[-0.0813,  0.2159, -0.4898, -0.1937, -0.3461],
          [-0.3994, -0.3074, -0.1513, -0.1011,  0.0380],
          [-0.0792,  0.1815, -0.2896,  0.3705, -0.4631],
          [ 0.1670, -0.0810,  0.0575, -0.3523, -0.2958],
          [ 0.2947,  0.4588, -0.1828,  0.1884,  0.3688]]],


        [[[ 0.3866, -0.4066, -0.4516, -0.3235,  0.3705],
          [-0.3935, -0.0773,  0.4486,  0.0325,  0.1880],
          [-0.1808,  0.1827,  0.3279, -0.4720,  0.2451],
          [ 0.4790,  0.2432, -0.2151,  0.2834, -0.3888],
          [-0.0511,  0.4003, -0.2022, -0.2079, -0.3625]]]],
       requires_grad=True)

In [16]:
np.random.seed(2)
layer = nn.Conv2d(1, 2, kernel_size=5, stride=(1, 1), padding=2, bias=False)
layer.apply(weights_init_kaiming)
torch_h = layer(torch_x)
print(torch_h.shape)
print(torch_h.sum().item(), "compare to tensorflow's result", -58702.617)

torch.Size([2, 1, 5, 5])
torch.Size([256, 2, 28, 28])
-58702.6015625 compare to tensorflow's result -58702.617


In [17]:
layer.weight

Parameter containing:
tensor([[[[-6.2712e-02, -4.6450e-01,  4.8659e-02, -6.3371e-02, -7.8023e-02],
          [-1.6624e-01, -2.8938e-01,  1.1686e-01, -1.9630e-01, -2.2846e-01],
          [ 1.1869e-01,  2.8553e-02, -3.5804e-01,  1.3304e-02, -3.0918e-01],
          [ 2.7957e-01,  3.4682e-01, -5.6467e-03,  3.3956e-01, -4.1186e-01],
          [ 5.1401e-03, -4.2593e-01, -7.0425e-02, -3.9532e-01, -3.6531e-01]]],


        [[[ 9.4791e-02, -2.6845e-01, -3.8511e-01, -2.7404e-01, -1.4714e-01],
          [-3.1562e-02, -2.9223e-01,  1.3757e-01, -1.6588e-02,  5.1309e-03],
          [-1.1082e-01,  2.8770e-01,  7.8388e-02, -3.3088e-01,  1.9670e-01],
          [ 4.5517e-01,  8.1922e-06,  3.8165e-01, -1.5519e-01,  6.5788e-02],
          [-7.0990e-02, -6.1975e-02,  2.7097e-01,  3.4885e-02,  4.4457e-01]]]],
       requires_grad=True)

In [18]:
np.random.seed(1)
layer = nn.Linear(784, 10)
layer.apply(weights_init_kaiming)

torch_h = layer(torch_x.view(256, -1))
print(torch_h.shape)
print(torch_h.sum().item(), "compare to tensorflow's result", -372.48087)

torch.Size([10, 784])
torch.Size([256, 10])
-372.48089599609375 compare to tensorflow's result -372.48087


In [19]:
layer.weight

Parameter containing:
tensor([[-0.0145,  0.0385, -0.0875,  ...,  0.0831, -0.0314,  0.0844],
        [ 0.0238, -0.0219,  0.0625,  ..., -0.0563,  0.0822,  0.0423],
        [-0.0420,  0.0142,  0.0797,  ...,  0.0347, -0.0618,  0.0197],
        ...,
        [-0.0104, -0.0067,  0.0168,  ..., -0.0227,  0.0015,  0.0317],
        [-0.0333, -0.0206,  0.0457,  ...,  0.0867, -0.0032,  0.0591],
        [ 0.0181,  0.0288, -0.0398,  ..., -0.0529, -0.0045,  0.0792]],
       requires_grad=True)

In [20]:
layer.bias

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)

In [21]:
torch_prob_vector = nfunc.softmax(torch_h, dim=1)

In [22]:
torch_prob_vector

tensor([[0.1658, 0.0800, 0.0848,  ..., 0.1385, 0.0740, 0.1402],
        [0.2034, 0.0764, 0.1040,  ..., 0.1530, 0.0461, 0.1414],
        [0.1279, 0.1226, 0.0992,  ..., 0.1597, 0.0540, 0.0869],
        ...,
        [0.1864, 0.1094, 0.1290,  ..., 0.1949, 0.0775, 0.0750],
        [0.0786, 0.1713, 0.0787,  ..., 0.1446, 0.0352, 0.0759],
        [0.0851, 0.1684, 0.0963,  ..., 0.2776, 0.0361, 0.0498]],
       grad_fn=<SoftmaxBackward>)

In [23]:
tf_prob_vector

<tf.Tensor: id=128, shape=(256, 10), dtype=float32, numpy=
array([[0.16578935, 0.07995356, 0.08477649, ..., 0.1385049 , 0.07403749,
        0.140184  ],
       [0.20340426, 0.07643472, 0.10398174, ..., 0.15296923, 0.04614347,
        0.14144777],
       [0.12792914, 0.12255355, 0.09924566, ..., 0.15965919, 0.05399203,
        0.08688193],
       ...,
       [0.18636437, 0.10938627, 0.12896058, ..., 0.19488415, 0.0774812 ,
        0.0749742 ],
       [0.07856708, 0.17134057, 0.07866888, ..., 0.1446415 , 0.03522116,
        0.0759176 ],
       [0.08513853, 0.16843319, 0.0963325 , ..., 0.27759305, 0.03611426,
        0.04978047]], dtype=float32)>